In [1]:
print("Preparing environment...")

from astropy import table as tbl
from astropy.io import fits
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import interpolate
from scipy import constants as c
import platform
from tqdm.auto import tqdm

# %matplotlib widget

print("Done.")

Preparing environment...
Done.


In [21]:
# Function to use appropriate OS directory structure
def GetDirStruct():
    match platform.system():
        case "Windows":
            dirPrefix = "D:/"
        case "Linux" | "Ubuntu":
            dirPrefix = "/mnt/d/"
        case "macOS" | "Darwin":
            dirPrefix = "/Volumes/Storage/"
        case _:
            raise Exception(f"OS not recognised: \"{platform.system()}\". Please define a custom switch inside GetDirStruct().")
    return dirPrefix


# Function to grab list of spectra in a directory
def GetFiles():
    dirPrefix = GetDirStruct()
    fileDir = dirPrefix + specFolder
    fileList = !bash -c "ls {fileDir}*1D.fits"
    fileList = [file.split('/')[-1] for file in fileList]
    return fileList, fileDir


# Function to import files to dictionary
def ImportFiles(fileList, fileDir):
    fileData = {}
    for file in tqdm(fileList, desc="Files"):
        fileRaw = fits.open(fileDir + file)
        fileFlux = fileRaw[1].data
        fileWave = fileRaw[9].data
        fileData[file.split('_')[0]] = tbl.Table([fileWave, fileFlux], names=("Wavelength", "Flux"))
        fileRaw.close()
    return fileData


# Function to plot the spectra
def PlotFiles(fileData, fileList):
    !bash -c "mkdir -p ../../Working_Directory/Apo_Phot_Utils/{outputFolder}"
    for file in tqdm(fileList, desc="Galaxies"):
        fileName = file.split('_')[0]
        plt.plot(fileData[fileName]["Wavelength"], fileData[fileName]["Flux"])
        plt.xscale('log')
        plt.savefig(f"../../Working_Directory/Apo_Phot_Utils/{outputFolder}{fileName}.png")
        plt.close()
    return

filterFolder = "Throughputs/nircam_throughputs/mean_throughputs/"
specFolder = "Spectra/HST_Medium/prism_clear_v1.5/prism_clear/"
outputFolder = "HST_Medium/prism_clear_v1.5/"
fileList, fileDir = GetFiles()
fileData = ImportFiles(fileList, fileDir)
PlotFiles(fileData, fileList)

Files:   0%|          | 0/577 [00:00<?, ?it/s]

Galaxies:   0%|          | 0/577 [00:00<?, ?it/s]

In [29]:
# Function to grab filter curves
def GrabFilter():
    dirPrefix = GetDirStruct()
    filterDir = dirPrefix + filterFolder
    filterList = !bash -c "ls {filterDir}"
    filterList = [file.split('/')[-1] for file in filterList]
    filterData = {}
    for filterFile in filterList:
        filterName = filterFile.split('_')[0]
        filterData[filterName] = tbl.Table.read(filterDir + filterFile, format="ascii")
        filterData[filterName]["Microns"] *= 10 ** (-6)
        filterData[filterName].rename_column("Microns", "Wavelength")
    return filterData

# Function to parse tables and set any rows with NaNs to zero, to avoid errors
def RemoveNaNs(tableObj):
    for col in tableObj.colnames:
        for x in range(0, len(tableObj[col]), 1):
            if np.isnan(tableObj[col][x]):
                print(f"NaN found! Position {x}, column {col}")
                tableObj[col][x] = 0
    return tableObj

# Function to interpolate datapoints
def InterpFunc(funcXs, funcYs):
    funcCube = interpolate.interp1d(funcXs, funcYs, kind='cubic')
    return funcCube

# Function to find grid overlap for convolutions
def FindGrid(funcXs, targetGrid, gridData):
    idxSorted = np.argsort(targetGrid)
    sortedGrid = np.array(targetGrid[idxSorted])
    idxLeft = np.searchsorted(sortedGrid, funcXs[0], side="right")
    if targetGrid[idxLeft-1] == funcXs[0]:
        idxLeft -= 1
    idxRight = np.searchsorted(sortedGrid, funcXs[-1], side="left")
    if targetGrid[idxRight] != funcXs[-1]:
        idxRight -= 1
    funcGrid = sortedGrid[idxLeft:idxRight]
    gridOverlap = np.array(gridData[idxSorted])[idxLeft:idxRight]
    return funcGrid, gridOverlap

# Function to integrate two functions on the same grid
def IntegFunc(funcConv, gridOverlap, funcGrid):
    funcInteg = np.trapz(funcConv * gridOverlap, x=funcGrid)
    return funcInteg

# Function to normalise the throughput to the correct zero-point
def NormFunc(funcConv, funcGrid, funcInteg):
    # norm_ref = 10**(48.6/(-2.5)) # reference zero magnitude
    normRef = 1 * u.Jy.to(u.W / ((u.m)**2 * u.Hz)) * 10**(-6) # reference flat-value in f_nu
    normGrid = normRef * c.c / funcGrid**2 * funcGrid # not squared, f_lambda in photon_space
    normValue = IntegFunc(funcConv, normGrid, funcGrid)
    funcNormed = funcInteg / normValue
    return normValue, funcNormed

# Function to manage overall convolution calculation
def ConvolveFunc(funcXs, funcYs, targetGrid, gridData):
    # Interpolate datapoints
    funcInterp = InterpFunc(funcXs, funcYs)
    # Find relevant convolution grid
    funcGrid, gridOverlap = FindGrid(funcXs, targetGrid, gridData)
    # Convolve onto grid
    funcConv = funcInterp(funcGrid)
    # Shift to photon space
    gridOverlap *= funcGrid
    # Perform integration
    funcInteg = IntegFunc(funcConv, gridOverlap, funcGrid)
    # Normalise the result
    normValue, funcNormed = NormFunc(funcConv, funcGrid, funcInteg)
    return normValue, funcNormed, funcInteg, funcConv, funcGrid, gridOverlap

print("Importing data...")

filterData = GrabFilter()

print("Cleaning filters...")
for filterFile in filterData.values():
    filterFile = RemoveNaNs(filterFile)
print("Cleaning spectra...")
Spectrum = RemoveNaNs(fileData['000416'])

print("Interpolating filters...")

filterNormValue, filterNormed, filterThrough, filterConv, filterGrid, spectrumFlux = ConvolveFunc(Filter["Wavelength"], Filter["Throughput"], Spectrum["Wavelength"], Spectrum["Flux"])

print("Done.")

Importing data...
Cleaning filters...
Cleaning spectra...
Interpolating filters...
Done.


In [19]:
filterNormed

-0.0004695418605066632

In [7]:
fileData.keys()

dict_keys(['000416', '000455', '000490', '000507', '000543', '000618', '001047', '001096', '001146', '001176', '001191', '001304', '001594', '001875', '001926', '001951', '002208', '002266', '002430', '002489', '002895', '003080', '003116', '003135', '003235', '003236', '003259', '003298', '003315', '003344', '003380', '003382', '003418', '003439', '003510', '004028', '004081', '004277', '004507', '004832', '004929', '005052', '005173', '005254', '005278', '005662', '005669', '005672', '005694', '005722', '005724', '005732', '005785', '005815', '005829', '005835', '005940', '005946', '005995', '006022', '006101', '006139', '006151', '006227', '006236', '006277', '006291', '006384', '006398', '006430', '006477', '006496', '006524', '006551', '006591', '006770', '006782', '006800', '006821', '006838', '006844', '006848', '006894', '006935', '006997', '007028', '007041', '007091', '007165', '007236', '007260', '007284', '007360', '007367', '007368', '007395', '007404', '007408', '007449',

In [ ]:
46.00287263466458 * 0.000899684595650764 * 0.79917

In [ ]:
Filter_normed

In [ ]:
plt.close("all")

# plt.plot(Spectrum["Wavelength"], Spectrum["Flux"])
# plt.plot(Filter["Wavelength"], Filter["Throughput"] * 10 **(-13))
plt.plot(Filter_grid, Filter_conv * 10 **(-13), color='green')
plt.plot(Filter_grid, Spectrum_flux, color='blue')
plt.plot(Filter_grid, Filter_conv * Spectrum_flux, color='red')